# SSD 物体检测: 用于实时处理的单目多框检测器
![avater](https://cdn-images-1.medium.com/max/800/1*zmQVpkThKXULL0mP86Dnng.png)

SSD设计用来进行实时物体检测。Faster R-CNN使用一个区域提取网络（RPN）构建一些边界框并且利用这些目标框对物体分类。在最高准确率下，整个处理过程是每秒7帧，这远远不能满足实时处理的需要。SSD通过取消RPN来加速处理过程。为了弥补准确率上的损失，SSD采取了一些改进方法如多尺度特征以及默认边界框。这些改进方法允许SSD使用较低解析度的图像就可以达到Faster R-CNN的精度，而且处理速度更快。从下图可以看出，SSD达到了实时处理的速度并且精度还强于Faster R-CNN。（精度的评价指标是mAP-预测平均精度）。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*rqGEyJKbKv3ecmjaMSiEtA.png)

## SSD
SSD包含两个部分：
- 提取特征图
- 应用卷积核检测物体
![avater](https://cdn-images-1.medium.com/max/800/1*aex5im2aYcsk4RVKUD4zeg.jpeg)

SSD使用**VGG16**来提取特征图。然后它使用Conv4_3的卷积层来检测物体。为了简化说明，我们假定Conv4_3是一个8x8的尺寸（实际上是38x38的）。对于每个单元（或者位置），它会预测4个结果。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*vNaiiFUVwCfzx1znKiFYYw.jpeg)
每个预测结果包含一个目标框以及21个类别的得分（其中一个是背景的类别），我们选择最高得分的类别作为该目标框的分类。Conv4_3总共会做出38x38x4个预测：每个单元4个预测，跟特征图的深度无关。正如期望中看到的，很多预测结果中不包含物体。SSD将这些不包含物体的归类为0。<br>
![avater](https://cdn-images-1.medium.com/max/1000/1*mvF9g_zH2DaQK2KgutndFg.jpeg)

### 做出包含多个边界框以及置信得分的预测过程称之为multibox。

## 用于物体检测的卷积预测器
SSD不使用代理RPN。相反，它采用了一种非常简单的方法。它使用**小卷积核**来预测位置和类别得分。在提取了特征图之后，SSD在每个单元上应用3x3的卷积核来做预测。（这些滤波器的计算过程跟常规CNN滤波器一致）每个滤波器输出一个25维的结果：其中21个类别的得分以及一个边界框的坐标。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*mGtRvk9g1PVWw6IkjEryuA.jpeg)
举例说明，在Conv4_3中，我们应用了4个3x3的滤波器将512的输入通道转变为25个输出通道。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*fxy_hGpFB8V9ZA3KYTGgfg.jpeg)

## 用于识别任务的多尺度特征图
![avater](https://cdn-images-1.medium.com/max/800/1*N-ZCvRQL9iwnlYTX5XM2Iw.jpeg)
首先，我们描述了SSD在单层上如何做物体检测的。事实上，它独立地使用多层（多尺度特征图）做物体检测。由于CNN逐级缩减空间尺寸，特征图的解析度也逐级减少。SSD使用较低解析度的层来检测较大尺度的物体。例如，这个4x4的特征图用于检测较大的物体。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*ZOeQglB3OQ3Qrc_L3r83dg.jpeg)
SSD在VGG16之后添加了6个辅助的卷积层。其中5个用于物体检测，在其中三个层中，我们每个位置做6个预测而不是4个预测。SSD利用6层总共做出8732个预测。<br>
![avater](https://cdn-images-1.medium.com/max/1000/1*up-gIJ9rPkHXUGRoqWuULQ.jpeg)
多尺度特征图极大地提升了准确率。这里准确率指在不同特征图层数下的准确率。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*WI2-RDIpx_mBYoKuvPJFDg.png)

## 默认边界框
默认边界框等同于Faster R-CNN中的**anchors**。<br>
我们如何预测目标框呢？就行深度学习一样，我们可以从随机猜测开始并且使用梯度下降来优化模型。然而，在初始训练过程中，模型会纠结于对什么形状（行人还是汽车）来做优化。经验表明较早期的训练是非常不稳定的。边界框预测在单个类别上的表现没问题但是对于多个类别就另当别论了。我们希望我们的初始边界框是差异化的，而不是彼此类似。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*7iwTsezrn-tSndx96twprA.jpeg)
如果我们的预测包含更多的形状，如下图所示。我们的模型可能会检测到更多的物体。这使得训练过程更加容易和稳定。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*wCHRhpMRx_SaIUfeO9kJZg.jpeg)
在实际中，边界框并不是任意的形状和尺寸。汽车会有相似的形状，行人也会有近似的纵横比，都接近于0.41。在用于自动驾驶的KITTI数据集中，边界框的宽度和高度分布是高度集中的。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*SEDBDJjvo8BABhR2MpVZ1Q.jpeg)
从概念上讲，真实值边界框可以通过聚类得到默认边界框（聚类的中心）。所以，不同于做随机猜测，我们可以基于这些默认边界框做预测。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*esSmI0UaMr-GrqkUGMg0hA.jpeg)
为了保持较低的复杂度，默认边界框可以提前手工选择来覆盖更多类的真实物体。SSD还保持着一个位置上默认4到6个边界框，每个边界框做一次预测。现在，不同于使用一个全局的坐标来定位边界框，而是预测相对于默认边界框的偏移量($\Delta cx,\Delta cy, w,h$)。<br>
对于每个特征图层，在其每个位置上共享一组默认边界框。但是不同的层使用不同的默认边界框组合以适应不同解析度上的物体识别。下图这4个绿色的边界框表示的是4个默认边界框。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*tDKlbaeHGSm_TwN02jLB8w.jpeg)

## 选择默认的边界框
![avater](https://cdn-images-1.medium.com/max/800/1*WKeaqu_u_efsMFQ6G7oKMQ.png)
默认的边界框是手动选择的。SSD在每个特征图层上定义了一个尺度值。从左边开始，Conv4_3有最小的尺度值0.2（有时是0.1），然后线性增长，到最右边的层时尺度值是0.9。将尺度值和纵横比结合起来，我们可以计算默认边界框的宽和高。对于做6个预测的层，SSD采用5个纵横比1,2,3,1/2以及1/3。然后默认边界框的宽和高计算公式如下：<br>
$$w=scale \cdot \sqrt{aspect\ ratio}$$
$$h=\frac{scale}{\sqrt{aspect\ ratio}}$$
然后SSD还加了一个附加的边界框，尺度是：<br>
$$scale=\sqrt{scale \cdot scale\ at\ next\ level}$$
同时纵横比是1。<br>
**YOLO在训练集上使用k-means聚类来得到默认边界框。**

## 匹配策略
SSD预测结果分为正例匹配或负例匹配。SSD仅使用正例匹配来计算边界框不匹配时的损失。如果相关的**默认边界框**(非预测边界框)相对于真实值的IoU大于0.5，匹配就是正的，否则就是负的。（IoU是联合区域的交并比，即交集比上并集）<br>
![avater](https://cdn-images-1.medium.com/max/800/1*3cr5JJXImPX2ZDVNFSvHrg.jpeg)
我们假设有三个默认的边界框。只有1和2（而不是3）的跟真实值的交并比大于0.5。因此只有边界框1和2是正例匹配。一旦我们找到了正例匹配，我们可以根据相关的预测边界框来计算损失。这个匹配策略能够很好的区分预测结果的真实值形状。<br>
这个匹配策略鼓励每个预测结果的形状趋近于相关的默认边界框。因此，我们的预测结果将更加多样性，在训练时更加稳定。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*gpxMpC_-HOZAYDYISkp2MA.jpeg)

## 多尺度特征图以及默认边界框
这里有一个SSD整合多尺度特征图和默认边界框的例子，用于检测不同尺度上的物体。下图中的狗跟4x4特征图中默认的边界框（红色）匹配，但是跟8x8特征图上的默认边界框都不匹配。图中的猫则在8x8的特征图中跟两个默认边界框（蓝色）匹配。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*-KVIXjvBO5m2MQZrzWx-wg.png)
高解析度的特征图用于检测较小的物体。第一个物体检测层conv4_3的尺寸是38x38，这相较于原始输入图已经缩小了很多。然而，SSD通常相较于其他检测方法在较小物体上表现较差。如果存在问题，我们可以通过获取更大解析度的图像来缓解。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*sZUWR2XgCAJ6AXM5NXYjNg.png)

## 损失函数
**定位损失**是真实边界框和预测边界框的不匹配程度。SSD只惩罚正例匹配的预测。我们希望正例匹配的预测结果跟真实值更加趋近。负例匹配可以忽略。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*cIE7bbicMOokWQ6w41I-NA.png)
置信损失是预测类别的损失。对于每个正例匹配预测，我们根据相关类别的置信得分来计算损失。对于负例匹配，我们根据类别“0”的置信得分来计算损失：类别“0”就是不包含物体。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*OTVm8L9RoAKtwl3XEQNkzA.png)
最终的损失公式为：<br>
$$L(x,c,l,g)=\frac{1}{N}{(L_{conf}(x,c)+\alpha L_{loc}{(x,l,g)})}$$
这里N是正例匹配的数量，$\alpha$是定位损失的权重。

## 负例挖掘
然而，我们会做出比实际物体数量多得多的预测。因此负例匹配会远多于正例匹配。这种类别不平衡会影响训练，此时我们训练的模型会更多地学习背景而不是检测物体。然而，SSD仍然需要负例采样来学习什么导致了较差的预测。所以，不同于使用全部负例，我们通过将这些负例按照置信损失排序。SSD选择最高损失的负例确保负例和正例的比例最多为3:1。这使得训练更加快速和稳定。

## 数据增强
数据增强对于提升准确率非常重要。可以通过翻转、裁剪和颜色变换来增强数据。为了处理不同物体尺寸和形状，每个训练图像通过下面三种方法之一随机采样：<br>
- 使用原始图
- 在IoU为0.1,0.3,0.5,0.7,0.9下采样一个图像块
- 随机采样图像块
采样图像块纵横比在1/2和2之间。然后变换为一个固定尺寸并且对一半的训练数据进行翻转。另外，我们可以应用图像扭曲。<br>
![avater](https://cdn-images-1.medium.com/max/1000/1*cvuV55GI_CaVul6cBSc35w.jpeg)
下面是应用数据增强后的提升结果：<br>
![avater](https://cdn-images-1.medium.com/max/800/1*QSXnP4qZsQp5R4eJu4sJKQ.jpeg)

## 预测时间
SSD为了覆盖所有位置、尺度和纵横比，做出了8732个预测，这远多于其他检测模型。然而，很多预测结果不包含任何物体。因此，任何类别置信得分小于0.01的预测将会被抛弃。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*upUWQsnSlAZD1VmeyjlreA.jpeg)

### 非极大值抑制(nms)
SSD使用非极大值抑制来去除冗余预测结果。SSD根据置信得分将预测结果排序。从最大的置信得分预测结果开始，SSD评估预测边界框跟排在前面的预测边界框之间是否有大于0.45的交并比。如果找到了，就忽略掉较低置信得分的边界框。最终，我们在一张图像上得到最多200个预测结果。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*aCMSCIUVkHrQI7QQEVnxew.png)

## 实验结果
模型使用SGD，在初始学习率为0.001，动量0.9，0.0005的权重衰减以及批数量32的条件下进行训练。使用一个在VOC2007 test数据集上使用一张Nvidia Titan X，SSD获得了59的FPS和74.3%的mAP。而Faster R-CNN的成绩是7FPS和73.2%的mAP，而YOLO的成绩是45FPS和63.4%的mAP。<br>
这里是不同方法的准确率比较。对于SSD，输入图像是300x300或512x512。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*O4rr3UD3PEzM9JacoOorNQ.png)
这里是不同方法的速度比较。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*upUWQsnSlAZD1VmeyjlreA.jpeg)

## 发现
这里是一些关键的发现：<br>
- SSD在小物体上的表现较Faster R-CNN差。在SSD中，小物体只能在较高解析度的特征图检测（最左边的层）。但是那些层往往包含较低级的特征，如边缘或颜色块，对于分类来说缺少信息。
- 默认目标框越多准确率越高，但是会带来较低的速度。
- 多尺度特征图可以提升不同尺度上的准确率。
- 设计更好的默认边界框有助于提升准确率。
- COCO数据集包含更小的物体。为了提升准确率，使用较小的默认边界框（从一个较低的尺度值如0.15开始）。
- SSD相较于R-CNN有更低的定位误差但是会有较高的分类误差。更高的分类误差可能是因为我们使用同样的边界框预测多个类别。
- SSD512相较于SSD300准确率多了2.5%，但是速度从59FPS变为22FPS。

## 结论
SSD是一个单目检测器。它没有使用代理RPN而是直接在特征图上对边界框和类别做预测。<br>
为了提高准确率，SSD改进如下：<br>
- 采用较小的卷积核来预测类别和相对于默认边界框的偏移量
- 引入了默认边界框来处理不同纵横比的物体
- 应用多尺度特征图来检测物体
SSD可以用端到端的训练来获得更高的精度。SSD为了覆盖不同的位置，尺度和纵横比做出了很多预测。基于以上提升手段，SSD可以在300x300的图像上得到一个非常有竞争力的表现。通过去掉RPN以及实用较低分别率的图像，模型可以以实时处理的速度达到超越Faster R-CNN的表现。

## 资源
- [Official Caffe code implementation](https://github.com/weiliu89/caffe/tree/ssd)
- [Paul Balanca's TensorFlow implementation](https://github.com/balancap/SSD-Tensorflow)